<a href="https://colab.research.google.com/github/matthewleechen/UKHistoricalPatents/blob/main/KPSTBreakthroughs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook provides code to implement the breakthrough score calculations we use in **300 Years of British Patents**.

In [ ]:
from datasets import load_dataset
import nltk
from nltk.corpus import stopwords as nltk_stopwords
import spacy
from gensim.parsing.preprocessing import STOPWORDS as gensim_stopwords
import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix, vstack, save_npz, load_npz
from collections import defaultdict, Counter
from tqdm.notebook import tqdm
import joblib
from joblib import Parallel, delayed
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:

def calculate_text_length(example):
    """
    gets total length of text across all pages
    """
    total_length = sum(len(page['page_text']) for page in example['full_text'])

    return {'text_length': total_length}



def filter_dataset_by_length(dataset, min_chars=1000):

    """
    keeps only dataset entries >= min_chars length
    """

    # add length column
    dataset = dataset.map(calculate_text_length)

    # keep entries above min length
    filtered_dataset = dataset.filter(lambda x: x['text_length'] >= min_chars)

    # show filtering results
    total = len(dataset)
    filtered = len(filtered_dataset)
    print(f"original size: {total}")
    print(f"filtered size: {filtered}")
    print(f"removed: {total - filtered}")

    return filtered_dataset


def load_all_stopwords():
    """
    loads stopwords from NLTK, spaCy, and gensim, and returns a combined list of plain strings
    """
    # load NLTK stopwords
    nltk_stops = set(nltk_stopwords.words('english'))

    # load spaCy model and extract stopwords
    nlp = spacy.load('en_core_web_lg')
    spacy_stops = nlp.Defaults.stop_words

    # combine with gensim stopwords
    combined = nltk_stops.union(spacy_stops).union(gensim_stopwords)

    # Return as a list of strings
    return list(combined)



# skip pre-1734 patents (before specifications were mandatory)
dataset_full = load_dataset(
    "gbpatentdata/300YearsOfBritishPatents",
    data_files="texts.jsonl.gz"
)

# skip pre-1734 patents (before specifications were mandatory)
dataset_subset_range = dataset_full["train"].filter(
    lambda x: 1734 <= x["year"] <= 1899
)

# keep only those with sufficient length
dataset_subset_years = filter_dataset_by_length(dataset_subset_range["train"], min_chars=1000)

In [ ]:
### 1) Vectorizer: Build vocabulary + backward-IDF, then chunked transform

class BackwardTFIDFVectorizer(BaseEstimator, TransformerMixin):
    """
    handles tf-bidf as in the KPST implementation. splits into fit() for vocab/bidf
    and transform() that works in chunks for memory efficiency
    """
    def __init__(self, min_df=5, max_df=0.5, stop_words=None, batch_size=1000):

        # min number of docs a term must appear in
        self.min_df = min_df

        # max proportion of docs a term can appear in
        self.max_df = max_df

        # words to filter out
        self.stop_words = set(stop_words) if stop_words else set()

        # will store {term: index} mapping
        self.vocabulary_ = {}

        # how many docs to process at once
        self.batch_size = batch_size


    def _compute_backward_idf_matrix(self, unique_years):
        """
        calculates backwards idf scores - higher weight for rare terms before time t
        bidf = log(total docs before t / (1 + docs with term before t))
        """
        print("Computing backward-IDF values...")

        # order years and map them to matrix indices
        sorted_years = np.sort(unique_years)
        year_to_idx = {year: idx for idx, year in enumerate(sorted_years)}

        # matrix to track term counts per year
        n_terms = len(self.vocabulary_)
        n_years = len(sorted_years)
        term_year_counts = np.zeros((n_terms, n_years), dtype=np.float32)

        print(f"Creating matrix of shape: {term_year_counts.shape} for term-year counts")

        # fill the count matrix - how many docs per year have each term
        for terms, year in tqdm(zip(self.doc_terms_, self.doc_years_),
                                total=len(self.doc_terms_),
                                desc="Counting term-year frequencies"):

            if year in year_to_idx:

                yr_idx = year_to_idx[year]

                for term in terms:

                    if term in self.vocabulary_:

                        term_idx = self.vocabulary_[term]

                        term_year_counts[term_idx, yr_idx] += 1


        print("Computing BIDF matrix...")

        # calculate backwards idf scores for each term-year pair
        bidf_matrix = np.zeros_like(term_year_counts)

        cumsum_terms = np.cumsum(term_year_counts, axis=1)  # cumulative sums across years

        patents_prior = np.arange(1, n_years + 1, dtype=np.float32)  # docs up to each year

        for i in range(n_years):

            # bidf formula: log(total_docs_before / docs_with_term_before)
            bidf_matrix[:, i] = np.log(
                patents_prior[i] / (1 + cumsum_terms[:, i])
            )

        return bidf_matrix, year_to_idx



    def fit(self, X, y=None):
        """
        builds vocab dictionary, filters rare/common terms, makes bidf matrix.
        needs years (y) to work!
        """
        if y is None:

            raise ValueError("Must provide 'y' as years to fit()!")

        print("Fitting BackwardTFIDFVectorizer...")

        # keep track of years and terms per doc
        self.doc_years_ = y
        self.doc_terms_ = []
        term_counts = defaultdict(int)

        # count how often each term appears
        for doc in tqdm(X, desc="Parsing docs for vocabulary"):
            # get unique terms minus stopwords
            tokens = set(t for t in doc.split() if t not in self.stop_words)

            self.doc_terms_.append(tokens)

            for t in tokens:
                term_counts[t] += 1


        # keep terms that aren't too rare or too common
        n_docs = len(X)
        min_docs = self.min_df if isinstance(self.min_df, int) else self.min_df * n_docs
        max_docs = self.max_df if isinstance(self.max_df, int) else self.max_df * n_docs

        filtered_terms = [
            term for term, count in term_counts.items()
            if min_docs <= count <= max_docs
        ]

        # create term->index mapping
        self.vocabulary_ = {term: idx for idx, term in enumerate(filtered_terms)}

        print(f"Vocabulary size after filtering: {len(self.vocabulary_)}")

        # make the bidf matrix
        unique_years = np.unique(y)
        self.bidf_matrix_, self.year_to_idx_ = self._compute_backward_idf_matrix(unique_years)

        # free up memory from the doc_terms_
        del self.doc_terms_

        return self



    def transform(self, X, y=None):
        """
        converts docs to tf-bidf vectors and normalizes them.
        returns sparse matrix [n_docs x n_terms]
        """

        if not hasattr(self, 'vocabulary_'):

            raise ValueError("Must call fit() before transform()!")

        if y is None:

            raise ValueError("Must provide years to transform()!")


        # prep for sparse matrix construction
        rows, cols, data = [], [], []
        n_docs = len(X)

        # process docs in batches to save memory
        for start_idx in range(0, n_docs, self.batch_size):

            end_idx = min(start_idx + self.batch_size, n_docs)

            batch_docs = X[start_idx:end_idx]

            batch_years = y[start_idx:end_idx]

            for local_idx, (doc, year) in enumerate(zip(batch_docs, batch_years)):

                global_idx = start_idx + local_idx

                # get term frequencies
                terms = doc.split()

                term_counter = Counter(t for t in terms if t in self.vocabulary_)

                doc_len = sum(term_counter.values())

                # if empty or year not in set, skip
                if doc_len == 0 or (year not in self.year_to_idx_):
                    continue
                year_idx = self.year_to_idx_[year]

                # build TF-BIDF for each term
                for term, count in term_counter.items():

                    term_idx = self.vocabulary_[term]

                    tf = count / doc_len

                    bidf = self.bidf_matrix_[term_idx, year_idx]

                    rows.append(global_idx)

                    cols.append(term_idx)

                    data.append(tf * bidf)

        # build the sparse matrix
        mat = sparse.csr_matrix(
            (data, (rows, cols)),
            shape=(n_docs, len(self.vocabulary_))
        )

        # L2 normalize each row
        row_norms = np.sqrt(mat.multiply(mat).sum(axis=1).A1)

        row_norms[row_norms == 0] = 1e-9

        mat = mat.multiply(1 / row_norms[:, np.newaxis])

        return mat



#### 2) Utility: Chunked transform -> store to disk

def chunked_transform_to_disk(vectorizer, texts, years, chunk_size=5000, out_file="tfidf_all.npz"):
    """
    transform all documents in chunks, build one large CSR matrix in memory, then store to disk
    """
    print(f"\nTransforming all docs in chunks of size {chunk_size}, then saving to disk: {out_file}")


    # transform each chunk
    all_chunks = []

    n_docs = len(texts)

    for start in tqdm(range(0, n_docs, chunk_size), desc="Building TF-IDF for entire dataset"):

        end = min(start + chunk_size, n_docs)

        mat_chunk = vectorizer.transform(texts[start:end], y=years[start:end])

        all_chunks.append(mat_chunk)

    # combine all chunk results
    tfidf_all = vstack(all_chunks, format='csr')

    sparse.save_npz(out_file, tfidf_all)

    print(f"TF-IDF matrix of shape {tfidf_all.shape} saved to {out_file}")



#### 3) Calculator Class: uses the disk-cached TF–IDF matrix

class BreakthroughScoreCalculator:

    """
    finds innovative patents by comparing similarity with past and future patents.
    high future / low past similarity = potential breakthrough
    """

    def __init__(
        self,
        stopwords,
        forward_horizon=10,
        backward_horizon=5,
        min_df=0.0,
        max_df=1.0,
        similarity_threshold=0.05,
        data_start_year=1734,
        data_end_year=1899,
        batch_size=1000,
        n_jobs=-1
    ):

        # store all the settings
        self.stopwords = stopwords
        self.forward_horizon = forward_horizon
        self.backward_horizon = backward_horizon
        self.min_df = min_df
        self.max_df = max_df
        self.similarity_threshold = similarity_threshold
        self.data_start_year = data_start_year
        self.data_end_year = data_end_year
        self.batch_size = batch_size
        self.n_jobs = n_jobs

        # calculate valid year range
        self.effective_start_year = self.data_start_year + self.backward_horizon

        self.effective_end_year = self.data_end_year - self.forward_horizon

        if self.effective_start_year >= self.effective_end_year:

            raise ValueError("Invalid effective year range")


    # checks if we have enough past/future data for this year
    def _is_within_effective_range(self, year):

        return self.effective_start_year <= year <= self.effective_end_year



    def _get_patent_text(self, patent_doc):
        """
        combines all pages into one string and lowercases it
        """
        pages = sorted(patent_doc['full_text'], key=lambda x: x['page_num'])

        text = ' '.join(page['page_text'] for page in pages)

        return text.lower()



    def compute_breakthrough_scores(self, dataset, tfidf_file="tfidf_all.npz"):
        """
        main pipeline:
         1) Filter dataset into effective range
         2) Fit BackwardTFIDFVectorizer
         3) Transform entire subset (chunked), store to disk
         4) Load stored matrix in memory
         5) For each year, slice out current docs + forward/backward docs, compute similarities
         6) Compute final metrics
        """
        print("Filtering dataset by effective year range...")

        # get valid patents
        patent_ids, years, texts = [], [], []

        for pat in tqdm(dataset, desc="Reading dataset"):

            y = pat["year"]

            if self._is_within_effective_range(y):

                patent_ids.append(pat["patent_id"])

                years.append(y)

                texts.append(self._get_patent_text(pat))

        if not texts:
            raise ValueError("No patents in the effective range!")



        patent_ids = np.array(patent_ids)

        years = np.array(years)

        texts = np.array(texts, dtype=object)

        # 1) fit vectorizer
        print(f"\nFitting on {len(texts)} patents total...")

        self.vectorizer_ = BackwardTFIDFVectorizer(
            min_df=self.min_df,
            max_df=self.max_df,
            stop_words=self.stopwords,
            batch_size=self.batch_size
        )

        self.vectorizer_.fit(texts, y=years)

        # 2) chunked transform all docs -> store to disk
        chunked_transform_to_disk(self.vectorizer_, texts, years,
                                  chunk_size=5000, out_file=tfidf_file)

        # 3) load the full TF-IDF matrix
        print(f"\nLoading TF-IDF matrix from disk: {tfidf_file}")
        tfidf_all = load_npz(tfidf_file)  # shape=(n_docs, vocab_size)

        # 4) year-based index
        year_to_indices = defaultdict(list)

        for i, y in enumerate(years):

            year_to_indices[y].append(i)

        # prepare arrays for sums and counts
        n = len(texts)
        fwd_sums = np.zeros(n, dtype=np.float64)
        fwd_counts = np.zeros(n, dtype=np.float64)
        bwd_sums = np.zeros(n, dtype=np.float64)
        bwd_counts = np.zeros(n, dtype=np.float64)

        # 5) for each year, slice row subsets
        unique_effective_years = sorted(year_to_indices.keys())

        for y in tqdm(unique_effective_years, desc="Year-based loop"):

            indices_current_year = year_to_indices[y]

            if not indices_current_year:
                continue

            current_matrix = tfidf_all[indices_current_year]

            # forward horizon
            fwd_indices = []

            for fwd_yr in range(y+1, y+1+self.forward_horizon):

                if fwd_yr in year_to_indices:

                    fwd_indices.extend(year_to_indices[fwd_yr])

            if fwd_indices:

                fwd_matrix = tfidf_all[fwd_indices]

                sim_fwd = current_matrix.dot(fwd_matrix.T).tocsr()

                # threshold
                sim_fwd.data[sim_fwd.data < self.similarity_threshold] = 0

                sim_fwd.eliminate_zeros()

                # sums, counts
                sums_local = np.array(sim_fwd.sum(axis=1)).ravel()

                counts_local = np.diff(sim_fwd.indptr)

                fwd_sums[indices_current_year] = sums_local

                fwd_counts[indices_current_year] = counts_local

            # backward horizon
            bwd_indices = []

            for bwd_yr in range(y - self.backward_horizon, y):

                if bwd_yr in year_to_indices:

                    bwd_indices.extend(year_to_indices[bwd_yr])

            if bwd_indices:

                bwd_matrix = tfidf_all[bwd_indices]

                sim_bwd = current_matrix.dot(bwd_matrix.T).tocsr()

                sim_bwd.data[sim_bwd.data < self.similarity_threshold] = 0

                sim_bwd.eliminate_zeros()

                sums_local = np.array(sim_bwd.sum(axis=1)).ravel()

                counts_local = np.diff(sim_bwd.indptr)

                bwd_sums[indices_current_year] = sums_local

                bwd_counts[indices_current_year] = counts_local

        # 6) final breakthrough metrics
        print("Computing final breakthrough metrics...")

        fwd_avgs = np.divide(
            fwd_sums, fwd_counts,
            out=np.zeros_like(fwd_sums), where=(fwd_counts > 0)
        )

        bwd_avgs = np.divide(
            bwd_sums, bwd_counts,
            out=np.zeros_like(bwd_sums), where=(bwd_counts > 0)
        )

        ############# NOTE: this section is different from KPST ###################


        bwd_avgs_safe = np.where(bwd_avgs == 0, 1e-9, bwd_avgs)

        # here the breakthrough score is an average, so centered on 1
        breakthrough_scores = fwd_avgs / bwd_avgs_safe


        ##### The KSPT implementation is to SUM over the fwd and bwd scores
        ##### To implement the exact KPST breakthrough scores, replace the two lines above with the two lines below:

        # bwd_sums_safe = np.where(bwd_sums == 0, 1e-9, bwd_sums)
        # breakthrough_scores = fwd_sums / bwd_sums_safe


        ############################################################################

        # build df
        results_df = pd.DataFrame({
            "patent_id": patent_ids,
            "year": years,
            "forward_similarity": fwd_avgs,
            "backward_similarity": bwd_avgs,
            "breakthrough_score": breakthrough_scores
        })
        return results_df




def main(dataset,
         forward_horizon=10,
         backward_horizon=5,
         min_df=0.0,
         max_df=1.0,
         similarity_threshold=0.05,
         data_start_year=1734,
         data_end_year=1899,
         batch_size=1000,
         n_jobs=-1,
         tfidf_file="tfidf_all.npz"):

    # define stopwords
    stopwords = load_all_stopwords()

    # instantiate breakthrough_scores calculator
    calculator = BreakthroughScoreCalculator(
        stopwords=stopwords,
        forward_horizon=forward_horizon,
        backward_horizon=backward_horizon,
        min_df=min_df,
        max_df=max_df,
        similarity_threshold=similarity_threshold,
        data_start_year=data_start_year,
        data_end_year=data_end_year,
        batch_size=batch_size,
        n_jobs=n_jobs
    )

    # compute breakthroughs
    results_df = calculator.compute_breakthrough_scores(
        dataset, tfidf_file=tfidf_file
    )
    return results_df

In [ ]:
# input list of patent dicts called `dataset_subset_years` from earlier load_dataset:
results = main(
    dataset=dataset_subset_years,
    forward_horizon=10,
    backward_horizon=5,
    min_df=0.0,
    max_df=1.0,
    similarity_threshold=0.05,
    batch_size=5000,
    n_jobs=-1,
    tfidf_file="my_tfidf_data_fh10_bh5_maxdf100.npz"
)
results.to_csv("breakthrough_scores_fh10_bh5_maxdf100.csv")

Filtering dataset by effective year range...


Reading dataset:  14%|█▍        | 43895/318472 [01:14<02:45, 1654.12it/s]

In [ ]:
results = main(
    dataset=dataset_subset_years,
    forward_horizon=5,
    backward_horizon=5,
    min_df=0.0,
    max_df=1.0,
    similarity_threshold=0.05,
    batch_size=5000,
    n_jobs=-1,
    tfidf_file="my_tfidf_data_fh5_bh5_maxdf100.npz"
)
results.to_csv("breakthrough_scores_fh5_bh5_maxdf100.csv")

In [ ]:
results = main(
    dataset=dataset_subset_years,
    forward_horizon=1,
    backward_horizon=5,
    min_df=0.0,
    max_df=1.0,
    similarity_threshold=0.05,
    batch_size=5000,
    n_jobs=-1,
    tfidf_file="my_tfidf_data_fh1_bh5_maxdf100.npz"
)
results.to_csv("breakthrough_scores_fh1_bh5_maxdf100.csv")

In [ ]:
results = main(
    dataset=dataset_subset_years,
    forward_horizon=20,
    backward_horizon=5,
    min_df=0.0,
    max_df=1.0,
    similarity_threshold=0.05,
    batch_size=5000,
    n_jobs=-1,
    tfidf_file="my_tfidf_data_fh20_bh5_maxdf100.npz"
)
results.to_csv("breakthrough_scores_fh20_bh5_maxdf100.csv")